In [1]:
#!pip install bubbly

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from bubbly.bubbly import bubbleplot 
from plotly.offline import init_notebook_mode, iplot

In [2]:
# def non_nan(df):
#     return [(df.index[row], df.columns[col]) for row in range(df.shape[0]) for col in range(df.iloc[row,:].shape[0]) if not np.isnan(df.iloc[row, col])]

def get_decade(year):
    return str(year)[:3]

# def transform_df(df, ref_df):
#     new_pop_df = pd.DataFrame(np.zeros(ref_df.shape), columns = ref_df.columns, index = ref_df.index)
#     for c in range(len(ref_df.columns)):
#         for r in range(len(ref_df)):
#             row = list(ref_df.index)[r]
#             col = list(ref_df.columns)[c]
#             #print(type(row), type(col))
#             pop = df[(df == row) & (df.Year == int(col))]
#             prop = ref_df[(ref_df.Entity == row) & (ref_df.Year == int(col))]
#             #print(len(pop), len(prop))
#             if len(pop) == 1 and len(prop) == 1:
#                 new_pop_df.iloc[r, c] = pop.iloc[0, 3] * (prop.iloc[0, 3]/100)
#     return new_pop_df

def transform_df(df):
    func = lambda x: x.split("_")[0]
    unique_cols = np.unique(list(map(func, list(df.columns))))
    num_rows = int(len(df) * len(unique_cols))
    num_cols = int(2 + len(df.columns)/len(unique_cols))
    
    new_df = pd.DataFrame(np.zeros((num_rows, num_cols)), columns = ["Index", "Year", "Lit", "Pop", "Sat"])
    
    j = 0
    for idx, i in enumerate(df.index):
        new_df["Index"].iloc[j:j + len(unique_cols)] = np.repeat(i, len(unique_cols))
        new_df["Year"].iloc[j:j + len(unique_cols)] = unique_cols
        j += len(unique_cols)
        
        
    dit = {"lit":2, "pop":3, "":4}
    for i in range(len(df)):
        for j in range(len(df.columns)):
            year = list(df.columns)[j].split("_")[0]
            index = list(df.index)[i]
            if len(list(df.columns)[j].split("_")) > 1:
                col = dit[list(df.columns)[j].split("_")[1]]
            else:
                col = dit[""]
            temp_df = new_df[(new_df.Year == year) & (new_df.Index == index)]
            new_df.iloc[list(temp_df.index)[0], col] = df.iloc[i, j]
    
    return new_df
    

In [3]:
#female_df_sub

In [4]:
continent_df = pd.read_csv("./countryContinent.csv", encoding="latin-1")

female_ed_df = pd.read_csv("./literacy_rate_adult_female_percent_of_females_ages_15_above.csv")
male_ed_df = pd.read_csv("./literacy_rate_adult_male_percent_of_males_ages_15_above.csv")

population_df = pd.read_csv("population-since-1800.csv")
female_pop_df = pd.read_csv("share-population-female.csv")

# Satisfaction rate by country over time
sat_df = pd.read_csv("satisfaction.csv")

In [5]:
sat_df

,Entity,Code,Year,Share of people who are happy (Eurobarometer 2017)
0,Albania,ALB,2014,58.685448
1,Albania,ALB,2015,62.037964
2,Albania,ALB,2016,59.073544
3,Austria,AUT,1996,93.234474
4,Austria,AUT,1997,83.727531
...,...,...,...,...
804,United Kingdom,GBR,2012,91.145721
805,United Kingdom,GBR,2013,89.813713
806,United Kingdom,GBR,2014,93.073212
807,United Kingdom,GBR,2015,92.553238


Setting index as country for female and male literacy rate DFs. 

In [6]:
female_ed_df = female_ed_df.set_index("country")
male_ed_df = male_ed_df.set_index("country")

# female = non_nan(female_ed_df)
# male = non_nan(male_ed_df)

# female.remove(("Afghanistan", "2010"))

In [14]:
empty_df = np.zeros(female_ed_df.shape)
empty_df[:] = np.nan

Creating new male and female population DFs to make info more accessible.

In [15]:
new_pop_df_f = pd.DataFrame(np.array(empty_df), columns = female_ed_df.columns, index = female_ed_df.index)
new_pop_df_m = pd.DataFrame(np.array(empty_df), columns = male_ed_df.columns, index = male_ed_df.index)

for c in range(len(female_ed_df.columns)):
    for r in range(len(female_ed_df)):
        row = list(female_ed_df.index)[r]
        col = list(female_ed_df.columns)[c]
        #print(type(row), type(col))
        pop = population_df[(population_df.Entity == row) & (population_df.Year == int(col))]
        prop = female_pop_df[(female_pop_df.Entity == row) & (female_pop_df.Year == int(col))]
        #print(len(pop), len(prop))
        if len(pop) == 1 and len(prop) == 1:
            new_pop_df_f.iloc[r, c] = pop.iloc[0, 3]*(prop.iloc[0, 3]/100)
            new_pop_df_m.iloc[r, c] = pop.iloc[0, 3]*(1-(prop.iloc[0, 3]/100))



In [16]:
#new_sat_df

Adding data from population_df to the new format of new_pop_df_(f/m) to make the population and literacy DFs more congruent.

In [17]:
new_sat_df = pd.DataFrame(np.array(empty_df), columns = female_ed_df.columns, index = female_ed_df.index)          
# for c in range(len(male_ed_df.columns)):
#     for r in range(len(male_ed_df)):
#         row = list(male_ed_df.index)[r]
#         col = list(male_ed_df.columns)[c]
#         pop = population_df[(population_df.Entity == row) & (population_df.Year == int(col))]
#         prop = female_pop_df[(female_pop_df.Entity == row) & (female_pop_df.Year == int(col))]
#         if len(pop) == 1 and len(prop) == 1:
            
            
count = 0
for c in range(len(female_ed_df.columns)):
    for r in range(len(female_ed_df)):
        row = list(female_ed_df.index)[r]
        col = list(female_ed_df.columns)[c]
        print(row, col)
        sat = sat_df[(sat_df.Entity == row) & (sat_df.Year == int(col))]
        #print(sat)
        if len(sat) == 1:
            count += 1
            new_sat_df.iloc[r, c] = sat.iloc[0, 3]

Aruba 1974
Afghanistan 1974
Angola 1974
Anguilla 1974
Albania 1974
Netherlands Antilles 1974
United Arab Emirates 1974
Argentina 1974
Armenia 1974
Antigua and Barbuda 1974
Azerbaijan 1974
Burundi 1974
Benin 1974
Burkina Faso 1974
Bangladesh 1974
Bulgaria 1974
Bahrain 1974
Bosnia and Herzegovina 1974
Belarus 1974
Belize 1974
Bolivia 1974
Brazil 1974
Brunei 1974
Bhutan 1974
Botswana 1974
Central African Republic 1974
Chile 1974
China 1974
Cote d'Ivoire 1974
Cameroon 1974
Congo, Dem. Rep. 1974
Colombia 1974
Comoros 1974
Cape Verde 1974
Costa Rica 1974
Cuba 1974
Cayman Islands 1974
Cyprus 1974
Dominican Republic 1974
Algeria 1974
Ecuador 1974
Egypt 1974
Eritrea 1974
Spain 1974
Estonia 1974
Ethiopia 1974
Gabon 1974
Georgia 1974
Ghana 1974
Guinea 1974
Gambia 1974
Guinea-Bissau 1974
Equatorial Guinea 1974
Greece 1974
Guatemala 1974
Guyana 1974
Honduras 1974
Croatia 1974
Haiti 1974
Hungary 1974
Indonesia 1974
India 1974
Iran 1974
Iraq 1974
Israel 1974
Italy 1974
Jamaica 1974
Jordan 1974
Kazakh

Yemen 1977
South Africa 1977
Zambia 1977
Zimbabwe 1977
Aruba 1978
Afghanistan 1978
Angola 1978
Anguilla 1978
Albania 1978
Netherlands Antilles 1978
United Arab Emirates 1978
Argentina 1978
Armenia 1978
Antigua and Barbuda 1978
Azerbaijan 1978
Burundi 1978
Benin 1978
Burkina Faso 1978
Bangladesh 1978
Bulgaria 1978
Bahrain 1978
Bosnia and Herzegovina 1978
Belarus 1978
Belize 1978
Bolivia 1978
Brazil 1978
Brunei 1978
Bhutan 1978
Botswana 1978
Central African Republic 1978
Chile 1978
China 1978
Cote d'Ivoire 1978
Cameroon 1978
Congo, Dem. Rep. 1978
Colombia 1978
Comoros 1978
Cape Verde 1978
Costa Rica 1978
Cuba 1978
Cayman Islands 1978
Cyprus 1978
Dominican Republic 1978
Algeria 1978
Ecuador 1978
Egypt 1978
Eritrea 1978
Spain 1978
Estonia 1978
Ethiopia 1978
Gabon 1978
Georgia 1978
Ghana 1978
Guinea 1978
Gambia 1978
Guinea-Bissau 1978
Equatorial Guinea 1978
Greece 1978
Guatemala 1978
Guyana 1978
Honduras 1978
Croatia 1978
Haiti 1978
Hungary 1978
Indonesia 1978
India 1978
Iran 1978
Iraq 1978

Uzbekistan 1981
Venezuela 1981
Vietnam 1981
Vanuatu 1981
Samoa 1981
Yemen 1981
South Africa 1981
Zambia 1981
Zimbabwe 1981
Aruba 1982
Afghanistan 1982
Angola 1982
Anguilla 1982
Albania 1982
Netherlands Antilles 1982
United Arab Emirates 1982
Argentina 1982
Armenia 1982
Antigua and Barbuda 1982
Azerbaijan 1982
Burundi 1982
Benin 1982
Burkina Faso 1982
Bangladesh 1982
Bulgaria 1982
Bahrain 1982
Bosnia and Herzegovina 1982
Belarus 1982
Belize 1982
Bolivia 1982
Brazil 1982
Brunei 1982
Bhutan 1982
Botswana 1982
Central African Republic 1982
Chile 1982
China 1982
Cote d'Ivoire 1982
Cameroon 1982
Congo, Dem. Rep. 1982
Colombia 1982
Comoros 1982
Cape Verde 1982
Costa Rica 1982
Cuba 1982
Cayman Islands 1982
Cyprus 1982
Dominican Republic 1982
Algeria 1982
Ecuador 1982
Egypt 1982
Eritrea 1982
Spain 1982
Estonia 1982
Ethiopia 1982
Gabon 1982
Georgia 1982
Ghana 1982
Guinea 1982
Gambia 1982
Guinea-Bissau 1982
Equatorial Guinea 1982
Greece 1982
Guatemala 1982
Guyana 1982
Honduras 1982
Croatia 1982
H

Senegal 1985
Singapore 1985
Sierra Leone 1985
El Salvador 1985
Serbia 1985
Sao Tome and Principe 1985
Suriname 1985
Slovenia 1985
Eswatini 1985
Seychelles 1985
Syria 1985
Chad 1985
Togo 1985
Thailand 1985
Tajikistan 1985
Turkmenistan 1985
Timor-Leste 1985
Tonga 1985
Trinidad and Tobago 1985
Tunisia 1985
Turkey 1985
Tanzania 1985
Uganda 1985
Ukraine 1985
Uruguay 1985
Uzbekistan 1985
Venezuela 1985
Vietnam 1985
Vanuatu 1985
Samoa 1985
Yemen 1985
South Africa 1985
Zambia 1985
Zimbabwe 1985
Aruba 1986
Afghanistan 1986
Angola 1986
Anguilla 1986
Albania 1986
Netherlands Antilles 1986
United Arab Emirates 1986
Argentina 1986
Armenia 1986
Antigua and Barbuda 1986
Azerbaijan 1986
Burundi 1986
Benin 1986
Burkina Faso 1986
Bangladesh 1986
Bulgaria 1986
Bahrain 1986
Bosnia and Herzegovina 1986
Belarus 1986
Belize 1986
Bolivia 1986
Brazil 1986
Brunei 1986
Bhutan 1986
Botswana 1986
Central African Republic 1986
Chile 1986
China 1986
Cote d'Ivoire 1986
Cameroon 1986
Congo, Dem. Rep. 1986
Colombia 198

Uruguay 1989
Uzbekistan 1989
Venezuela 1989
Vietnam 1989
Vanuatu 1989
Samoa 1989
Yemen 1989
South Africa 1989
Zambia 1989
Zimbabwe 1989
Aruba 1990
Afghanistan 1990
Angola 1990
Anguilla 1990
Albania 1990
Netherlands Antilles 1990
United Arab Emirates 1990
Argentina 1990
Armenia 1990
Antigua and Barbuda 1990
Azerbaijan 1990
Burundi 1990
Benin 1990
Burkina Faso 1990
Bangladesh 1990
Bulgaria 1990
Bahrain 1990
Bosnia and Herzegovina 1990
Belarus 1990
Belize 1990
Bolivia 1990
Brazil 1990
Brunei 1990
Bhutan 1990
Botswana 1990
Central African Republic 1990
Chile 1990
China 1990
Cote d'Ivoire 1990
Cameroon 1990
Congo, Dem. Rep. 1990
Colombia 1990
Comoros 1990
Cape Verde 1990
Costa Rica 1990
Cuba 1990
Cayman Islands 1990
Cyprus 1990
Dominican Republic 1990
Algeria 1990
Ecuador 1990
Egypt 1990
Eritrea 1990
Spain 1990
Estonia 1990
Ethiopia 1990
Gabon 1990
Georgia 1990
Ghana 1990
Guinea 1990
Gambia 1990
Guinea-Bissau 1990
Equatorial Guinea 1990
Greece 1990
Guatemala 1990
Guyana 1990
Honduras 1990
C

Chad 1994
Togo 1994
Thailand 1994
Tajikistan 1994
Turkmenistan 1994
Timor-Leste 1994
Tonga 1994
Trinidad and Tobago 1994
Tunisia 1994
Turkey 1994
Tanzania 1994
Uganda 1994
Ukraine 1994
Uruguay 1994
Uzbekistan 1994
Venezuela 1994
Vietnam 1994
Vanuatu 1994
Samoa 1994
Yemen 1994
South Africa 1994
Zambia 1994
Zimbabwe 1994
Aruba 1995
Afghanistan 1995
Angola 1995
Anguilla 1995
Albania 1995
Netherlands Antilles 1995
United Arab Emirates 1995
Argentina 1995
Armenia 1995
Antigua and Barbuda 1995
Azerbaijan 1995
Burundi 1995
Benin 1995
Burkina Faso 1995
Bangladesh 1995
Bulgaria 1995
Bahrain 1995
Bosnia and Herzegovina 1995
Belarus 1995
Belize 1995
Bolivia 1995
Brazil 1995
Brunei 1995
Bhutan 1995
Botswana 1995
Central African Republic 1995
Chile 1995
China 1995
Cote d'Ivoire 1995
Cameroon 1995
Congo, Dem. Rep. 1995
Colombia 1995
Comoros 1995
Cape Verde 1995
Costa Rica 1995
Cuba 1995
Cayman Islands 1995
Cyprus 1995
Dominican Republic 1995
Algeria 1995
Ecuador 1995
Egypt 1995
Eritrea 1995
Spain 19

Cameroon 1999
Congo, Dem. Rep. 1999
Colombia 1999
Comoros 1999
Cape Verde 1999
Costa Rica 1999
Cuba 1999
Cayman Islands 1999
Cyprus 1999
Dominican Republic 1999
Algeria 1999
Ecuador 1999
Egypt 1999
Eritrea 1999
Spain 1999
Estonia 1999
Ethiopia 1999
Gabon 1999
Georgia 1999
Ghana 1999
Guinea 1999
Gambia 1999
Guinea-Bissau 1999
Equatorial Guinea 1999
Greece 1999
Guatemala 1999
Guyana 1999
Honduras 1999
Croatia 1999
Haiti 1999
Hungary 1999
Indonesia 1999
India 1999
Iran 1999
Iraq 1999
Israel 1999
Italy 1999
Jamaica 1999
Jordan 1999
Kazakhstan 1999
Kenya 1999
Kyrgyz Republic 1999
Cambodia 1999
Kuwait 1999
Lao 1999
Lebanon 1999
Liberia 1999
Libya 1999
Sri Lanka 1999
Lesotho 1999
Lithuania 1999
Latvia 1999
Macao, China 1999
Morocco 1999
Moldova 1999
Madagascar 1999
Maldives 1999
Mexico 1999
North Macedonia 1999
Mali 1999
Malta 1999
Myanmar 1999
Montenegro 1999
Mongolia 1999
Mozambique 1999
Mauritania 1999
Mauritius 1999
Malawi 1999
Malaysia 1999
Namibia 1999
Niger 1999
Nigeria 1999
Nicaragua 

Venezuela 2003
Vietnam 2003
Vanuatu 2003
Samoa 2003
Yemen 2003
South Africa 2003
Zambia 2003
Zimbabwe 2003
Aruba 2004
Afghanistan 2004
Angola 2004
Anguilla 2004
Albania 2004
Netherlands Antilles 2004
United Arab Emirates 2004
Argentina 2004
Armenia 2004
Antigua and Barbuda 2004
Azerbaijan 2004
Burundi 2004
Benin 2004
Burkina Faso 2004
Bangladesh 2004
Bulgaria 2004
Bahrain 2004
Bosnia and Herzegovina 2004
Belarus 2004
Belize 2004
Bolivia 2004
Brazil 2004
Brunei 2004
Bhutan 2004
Botswana 2004
Central African Republic 2004
Chile 2004
China 2004
Cote d'Ivoire 2004
Cameroon 2004
Congo, Dem. Rep. 2004
Colombia 2004
Comoros 2004
Cape Verde 2004
Costa Rica 2004
Cuba 2004
Cayman Islands 2004
Cyprus 2004
Dominican Republic 2004
Algeria 2004
Ecuador 2004
Egypt 2004
Eritrea 2004
Spain 2004
Estonia 2004
Ethiopia 2004
Gabon 2004
Georgia 2004
Ghana 2004
Guinea 2004
Gambia 2004
Guinea-Bissau 2004
Equatorial Guinea 2004
Greece 2004
Guatemala 2004
Guyana 2004
Honduras 2004
Croatia 2004
Haiti 2004
Hungar

Senegal 2007
Singapore 2007
Sierra Leone 2007
El Salvador 2007
Serbia 2007
Sao Tome and Principe 2007
Suriname 2007
Slovenia 2007
Eswatini 2007
Seychelles 2007
Syria 2007
Chad 2007
Togo 2007
Thailand 2007
Tajikistan 2007
Turkmenistan 2007
Timor-Leste 2007
Tonga 2007
Trinidad and Tobago 2007
Tunisia 2007
Turkey 2007
Tanzania 2007
Uganda 2007
Ukraine 2007
Uruguay 2007
Uzbekistan 2007
Venezuela 2007
Vietnam 2007
Vanuatu 2007
Samoa 2007
Yemen 2007
South Africa 2007
Zambia 2007
Zimbabwe 2007
Aruba 2008
Afghanistan 2008
Angola 2008
Anguilla 2008
Albania 2008
Netherlands Antilles 2008
United Arab Emirates 2008
Argentina 2008
Armenia 2008
Antigua and Barbuda 2008
Azerbaijan 2008
Burundi 2008
Benin 2008
Burkina Faso 2008
Bangladesh 2008
Bulgaria 2008
Bahrain 2008
Bosnia and Herzegovina 2008
Belarus 2008
Belize 2008
Bolivia 2008
Brazil 2008
Brunei 2008
Bhutan 2008
Botswana 2008
Central African Republic 2008
Chile 2008
China 2008
Cote d'Ivoire 2008
Cameroon 2008
Congo, Dem. Rep. 2008
Colombia 200

In [19]:
count

202

The above operation makes the merge the preferrable method to combine info into a single DF for each gender.

In [23]:
female_ed_df = female_ed_df.merge(new_pop_df_f, left_index = True, right_index = True, suffixes = ("_lit", "_pop"))
male_ed_df = male_ed_df.merge(new_pop_df_m, left_index = True, right_index = True, suffixes = ("_lit", "_pop"))
female_ed_df = female_ed_df.merge(new_sat_df, left_index = True, right_index = True)
male_ed_df = male_ed_df.merge(new_sat_df, left_index = True, right_index = True)

We also need to merge the literacy DFs with some geographical info (continent and subregion classifications) about the countries.

In [24]:
female_ed_df = female_ed_df.merge(continent_df, left_index = True, right_on = "country", how = "left")
male_ed_df = male_ed_df.merge(continent_df, left_index = True, right_on = "country", how = "left")
#new_sat_df = new_sat_df.merge(continent_df, left_index = True, right_on = "country", how = "left")

In [25]:
#new_sat_df

We drop some unneccessary columns to clean up our DFs.

In [26]:
female_ed_df = female_ed_df.set_index("country")
male_ed_df = male_ed_df.set_index("country")
#new_sat_df = new_sat_df.set_index("country")
female_ed_df = female_ed_df.drop(['code_2', 'code_3', 'country_code', 'iso_3166_2', 'sub_region_code', 'region_code'], axis = 1)
male_ed_df = male_ed_df.drop(['code_2', 'code_3', 'country_code', 'iso_3166_2', 'sub_region_code', 'region_code'], axis = 1)
#new_sat_df = new_sat_df.drop(['code_2', 'code_3', 'country_code', 'iso_3166_2', 'sub_region_code', 'region_code'], axis = 1)

In [14]:
#female_ed_df.iloc[4,37]

We group the data in the DFs by subregion because we don't have enough data to analyze each country separately. We could do the same with continents as well.

In [15]:
for i in range(len(female_ed_df)):
    #print(female_ed_df.iloc[i,37])
    if not isinstance(female_ed_df.iloc[i,74], str):
        female_ed_df.iloc[i,111] = "Other"
    if not isinstance(male_ed_df.iloc[i,74], str):
        male_ed_df.iloc[i,111] = "Other"

female_df_sub = female_ed_df.groupby("sub_region").mean()
male_df_sub = male_ed_df.groupby("sub_region").mean()

In [16]:
# female_df_sub = female_df_sub.merge(sat_df_sub, left_index = True, right_index = True)
# male_df_sub = male_df_sub.merge(sat_df_sub, left_index = True, right_index = True)

In [103]:
new_sat_df

,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
country,,,,,,,,,,,,,,,,,,,,,
Aruba,2.946300e+04,29505.0,2.947500e+04,2.937700e+04,2.926900e+04,2.922300e+04,29291.0,2.951700e+04,2.988600e+04,3.029500e+04,...,44643.0,45486.0,46310.0,47028.0,47574.0,47935.0,48140.0,48253.0,48340.0,48465.0
Afghanistan,6.308583e+06,6446273.0,6.573732e+06,6.689144e+06,6.776023e+06,6.813205e+06,6788273.0,6.698730e+06,6.557673e+06,6.388704e+06,...,11117754.0,11642106.0,12214634.0,12763726.0,13239684.0,13623896.0,13938970.0,14225725.0,14544055.0,14935756.0
Angola,3.461420e+06,3586574.0,3.709688e+06,3.832864e+06,3.957984e+06,4.088285e+06,4226099.0,4.371520e+06,4.523437e+06,4.681598e+06,...,8423958.0,8704663.0,8999242.0,9310252.0,9639470.0,9987680.0,10354340.0,10738130.0,11137010.0,11549277.0
Anguilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,1.210869e+06,1237893.0,1.265219e+06,1.292864e+06,1.320788e+06,1.348940e+06,1377288.0,1.404951e+06,1.431805e+06,1.459310e+06,...,1586178.0,1582936.0,1574841.0,1563966.0,1551975.0,1538721.0,1523893.0,1508919.0,1495558.0,1485068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samoa,7.623800e+04,76801.0,7.734600e+04,7.785900e+04,7.837500e+04,7.893100e+04,79536.0,8.023500e+04,8.100200e+04,8.179600e+04,...,91319.0,91762.0,92202.0,92659.0,93129.0,93625.0,94138.0,94690.0,95294.0,95979.0
Yemen,3.220855e+06,3291854.0,3.380275e+06,3.485257e+06,3.604620e+06,3.734776e+06,3873129.0,4.020360e+06,4.177736e+06,4.343961e+06,...,9035617.0,9298002.0,9569772.0,9849046.0,10134482.0,10426557.0,10726485.0,11034174.0,11349365.0,11671585.0
South Africa,1.223440e+07,12555653.0,1.287449e+07,1.319275e+07,1.351744e+07,1.385833e+07,14221869.0,1.461132e+07,1.502301e+07,1.544795e+07,...,22468020.0,22752951.0,23032769.0,23315480.0,23607459.0,23910124.0,24223890.0,24552622.0,24900459.0,25269501.0


In [100]:
# for i in range(74, 111):
#     cols = list(female_df_sub.columns)
#     x = cols[i] + "_lit"
#     y = cols[i]
#     size = cols[i] + "_pop"
#     plt.figure(i)
    
trans_f = transform_df(female_df_sub)
trans_f["Gender"] = np.repeat("F", len(trans_f))
trans_m = transform_df(male_df_sub)
trans_m["Gender"] = np.repeat("M", len(trans_m))
frames = [trans_f, trans_m]
both_gen_trans = pd.concat(frames)
both_gen_trans["Year"] = pd.Series(map(int, both_gen_trans["Year"]))
both_gen_trans = both_gen_trans.dropna()
both_gen_trans.index = range(len(both_gen_trans))

subregions = np.unique(both_gen_trans["Index"])
for sub in subregions:
    try:
        temp_df = both_gen_trans[both_gen_trans.Index == sub]
        temp_df = temp_df.drop(["Index"], axis = 1)
        figure = bubbleplot(dataset=temp_df, x_column='Sat', y_column='Lit',
            bubble_column='Gender', time_column='Year', size_column='Pop', color_column='Gender', 
            x_title="Satisfaction Rate", y_title="Literacy Rate", title=sub,
             scale_bubble=3, height=650)
        iplot(figure, config={'scrollzoom': True})
    except:
        print("Unable to plot for " + sub)
    




Unable to plot for Micronesia


In [102]:
both_gen_trans

,Index,Year,Lit,Pop,Sat,Gender
0,Caribbean,1974,NaN,1.572989e+06,1.572989e+06,F
1,Caribbean,1975,NaN,1.600015e+06,1.600015e+06,F
2,Caribbean,1976,NaN,1.626357e+06,1.626357e+06,F
3,Caribbean,1977,NaN,1.652131e+06,1.652131e+06,F
4,Caribbean,1978,NaN,1.677469e+06,1.677469e+06,F
...,...,...,...,...,...,...
698,Western Asia,2006,95.583333,3.900099e+06,3.900099e+06,M
699,Western Asia,2007,93.100000,4.040672e+06,4.040672e+06,M
700,Western Asia,2008,97.100000,4.188472e+06,4.188472e+06,M
701,Western Asia,2009,95.180000,4.340344e+06,4.340344e+06,M


Uncommenting the piece of code above and commenting the other half of the code cell will plot by-country data instead of by-subregion data.

In [ ]:
# =============================================================================
# for j in range(len(female_ed_df)):
#     
#     new_df0 = female_ed_df.iloc[j,:37]
#     new_df0 = new_df0[new_df0.isnull()==False]
#     new_df1 = male_ed_df.iloc[j,:37]
#     new_df1 = new_df1[new_df1.isnull()==False]
#     
#     years_female = list(pd.Series(new_df0.index).apply(int))
#     literacy_female = list(new_df0)
#     years_male = list(pd.Series(new_df1.index).apply(int))
#     literacy_male = list(new_df1)
#     plt.figure(j)
#     
#     plt.plot(years_female, literacy_female, c="red", label = "Female")
#     plt.plot(years_male, literacy_male, c="blue", label = "Male")
#     plt.legend()
#     plt.title(female_ed_df.index[j])
# =============================================================================
func1 = lambda x: x.split("_")[0] + "_pop"
func2 = lambda x: x/1000 if not np.isnan(x) else x
for j in range(len(female_df_sub)):
    
    new_df0 = female_df_sub.iloc[j, :37]
    new_df0 = new_df0[new_df0.isnull()==False]
    new_df01 = female_df_sub.iloc[j, 37:74].get(list(map(func1, list(new_df0.index))))
    #print(new_df01)
    
    new_df1 = male_df_sub.iloc[j, :37]
    new_df1 = new_df1[new_df1.isnull()==False]
    new_df11 = male_df_sub.iloc[j, 37:74].get(list(map(func1, list(new_df1.index))))
    #print(new_df11)
    
    
    if isinstance(new_df01, pd.Series) and isinstance(new_df11, pd.Series):
        mx = max([max(new_df11), max(new_df01)])
        new_df01 = (10*(new_df01/mx))**3
        new_df11 = (10*(new_df11/mx))**3
    
    #print(new_df01)
    #print(new_df11)
    
    years_female = list(pd.Series(new_df0.index).apply(lambda x: int(x.split("_")[0])))
    literacy_female = list(new_df0)
    years_male = list(pd.Series(new_df1.index).apply(lambda x: int(x.split("_")[0])))
    literacy_male = list(new_df1)
    
    #print(len(years_female), len(list(new_df01)))
    if isinstance(new_df11, pd.Series) and isinstance(new_df01, pd.Series):
        plt.figure(j)
    
        plt.scatter(years_female, literacy_female, s = list(new_df01), c="red", label = "Female", alpha = 0.5)
        plt.scatter(years_male, literacy_male, s = list(new_df11), c="blue", label = "Male", alpha = 0.5)
        plt.plot(years_female, literacy_female, c="red", alpha = 0.5)
        plt.plot(years_male, literacy_male, c="blue", alpha = 0.5)
    #plt.xticks(rotation=-90)
        plt.legend()
        plt.title(female_df_sub.index[j])

In [ ]:
from gapminder import gapminder

df = pd.read_csv("population-since-1800.csv")
df

In [ ]:
!pip install gapminder